In [ ]:
# Pytorch를 활용한 자연어 처리 (NLP)
# 출처 : https://zzsza.github.io/data/2018/03/10/nlp-python/
# 사람은 문맥을 통해 각 단어의 유사함과 관계를 쉽게 추론할 수 있음 컴퓨터에게 넣기 위해선 컴퓨터가 이해할 수 있도록
# 변형해줘야 합니다. 이 과정을 Word Representation이라고 합니다.

In [ ]:
# Word Representation
# Idea / Thing을 여러 Symbol로 표현할 수 있습니다. 여기서 각 단어간 관계도 알 수 있다.

# WordNet
# * 단어 의미를 그래프 형태로 출력
# * 사람이 직접 구축해야 한다(비용이 비싸다)
# * 신조어의 의미를 이해 못함
# * 뉘앙스를 놓치기가 쉽다
# * 주관적이다
# * 단어 간의 정확한 유사도 계산이 어렵다(얼마나 유사한지) 그래프상 몇 노드가 연결되는지 정도만 알 수 있음


In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import nltk
import torchtext
from konlpy.tag import Kkma

tagger = Kkma()
import gensim

torchtext.vocab.pretrained_aliases

corpus = open('data/corpus.txt','r',encoding="utf-8").readlines()
corpus = [c[:-1] for c in corpus]

tokenized = [tagger.morphs(c) for c in corpus]

model = gensim.models.Word2Vec(tokenized, size=15, window=5, min_count=2, workers=4)

model.most_similar("토치")

model.wv.save_word2vec_format("data/word_vector_sample.bin",binary=True) # 저장

pretrained_vectors_model = gensim.models.KeyedVectors.load_word2vec_format("data/word_vector_sample.bin",binary=True)

pretrained_vectors_model['토치']

vocab = list(pretrained_vectors_model.vocab.keys()) # Word2Vec에서 사용한 vocab

pretrained_vectors=[]
for vo in vocab:
    pretrained_vectors.append(pretrained_vectors_model[vo])
    
pretrained_vectors = np.vstack(pretrained_vectors)

pretrained_vectors.shape

# Init embedding matrix
class MyModel(nn.Module):
    def __init__(self,vocab_size,embed_size):
        super(MyModel,self).__init__()
        
        self.embed = nn.Embedding(vocab_size,embed_size)
        
    def init_embed(self,pretrained_vectors):
        self.embed.weight.data = torch.from_numpy(pretrained_vectors).float()
    
    def forward(self,inputs):
        return self.embed(inputs)

model = MyModel(len(vocab),15)
model.embed.weight
model.init_embed(pretrained_vectors)
model.embed.weight

ModuleNotFoundError: ignored